In [6]:
import os
import pandas as pd
import torch
import numpy as np
import scipy.io as sio
from scipy.stats import zscore
from scipy.io import loadmat
from sklearn.metrics import mutual_info_score , normalized_mutual_info_score
from statsmodels.tsa.stattools import grangercausalitytests
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm  # Progress bar
import logging

# ----------------------------
# Setup logging for error handling
# ----------------------------
logging.basicConfig(filename='tensor_processing.log', level=logging.ERROR)

# ----------------------------
#  Discretization for Mutual Information
# ----------------------------
def discretize_signals(signals, num_bins=10):
    """
    Discretize continuous signals into a fixed number of bins for MI computation.
    """
    discretized_signals = np.zeros_like(signals)
    for i in range(signals.shape[1]):  # Loop over each ROI (column)
        discretized_signals[:, i] = np.digitize(
            signals[:, i],
            bins=np.histogram_bin_edges(signals[:, i], bins=num_bins)
        )
    return discretized_signals

# ----------------------------
#  Mutual Information
# ----------------------------
def compute_mutual_information_matrix(signals, num_bins=10):
    """
    Compute the mutual information matrix for discretized signals.
    """
    discretized_signals = discretize_signals(signals, num_bins)
    num_rois = discretized_signals.shape[1]
    mi_matrix = np.zeros((num_rois, num_rois))

    for i in range(num_rois):
        for j in range(num_rois):
            mi_matrix[i, j] = normalized_mutual_info_score(
                discretized_signals[:, i],
                discretized_signals[:, j]
            )
    return mi_matrix

# ----------------------------
#  Granger Causality Helpers
# ----------------------------
def granger_for_pair(args):
    """
    Granger causality function with error handling and logging.
    Returns -log(p-value) for the specified lag or 0 on error.
    """
    signals, i, j, max_lag = args
    if i != j:
        try:
            test_result = grangercausalitytests(signals[:, [i, j]], max_lag, verbose=False)
            p_value = test_result[max_lag][0]['ssr_ftest'][1]
            return i, j, -np.log(p_value) if p_value > 0 else 0
        except Exception as e:
            logging.error(f"Error computing Granger causality for pair {i}, {j}: {e}")
            return i, j, 0
    return i, j, 0

def compute_granger_causality_matrix_parallel(signals, max_lag=1, num_workers=4):
    """
    Compute Granger Causality matrix in parallel using ProcessPoolExecutor.
    """
    num_rois = signals.shape[1]
    gc_matrix = np.zeros((num_rois, num_rois))

    # Prepare the arguments for parallel computation
    args = [(signals, i, j, max_lag) for i in range(num_rois) for j in range(num_rois)]

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(granger_for_pair, arg) for arg in args]
        for future in as_completed(futures):
            i, j, value = future.result()
            gc_matrix[i, j] = value

    return gc_matrix

# ----------------------------
#  Pearson Correlation
# ----------------------------
def compute_correlation_matrix(signals):
    """
    Compute the Pearson correlation matrix (z-scored by column).
    """
    zscored_signals = zscore(signals, axis=0)
    return np.corrcoef(zscored_signals, rowvar=False)  # rowvar=False => correlate columns

# ----------------------------
#  Dynamic Connectivity (Sliding-Window Variability)
# ----------------------------
def compute_dynamic_connectivity_variability(signals, win_points=20, step=10):
    """
    1) Splits the signals into overlapping windows.
    2) Computes absolute Pearson correlation for each window.
    3) Computes Mdiff_mean: mean of absolute differences of consecutive correlation matrices.
       Final shape => (Nrois, Nrois).
    """
    # Ensure shape is (Time, ROIs) to match original snippet logic:
    #   In the snippet, Mdata is shape (Nrois, Ndata), but the loop does "Mdata[:, start:start+winPoints]".
    #   So let's just transpose our signals to match that dimension usage.
    #   Or we can adapt the code. We'll adapt the code to the shape (Ndata, Nrois).

    signals_T = signals  # shape (Ndata, Nrois) already (the code uses signals[:, some_roi])

    Ndata, Nrois = signals_T.shape
    Nwindows = (Ndata - win_points) // step + 1

    # 3D matrix for storing windowed correlation
    Mdata_Pearson_3D = np.zeros((Nrois, Nrois, Nwindows))

    for idx in range(Nwindows):
        start = idx * step
        xwin = signals_T[start:start + win_points, :]  # shape (win_points, Nrois)
        # Corrcoef wants shape (Nrois, Time) => so we transpose xwin
        Mdata_Pearson_3D[:, :, idx] = np.abs(np.corrcoef(xwin.T))

    # Differences between successive correlation matrices
    Mdiff_3D = np.abs(np.diff(Mdata_Pearson_3D, axis=2))  # shape => (Nrois, Nrois, Nwindows-1)

    # Mean across the third dimension
    Mdiff_mean = np.mean(Mdiff_3D, axis=2)  # shape => (Nrois, Nrois)

    return Mdiff_mean

# ----------------------------
#  Combine 4 Measures into 3D/4D Tensor
# ----------------------------
def combine_matrices_to_tensor(corr_matrix, mi_matrix, gc_matrix, dynamic_matrix):
    """
    Stack 4 matrices along a new axis => shape: (4, Nrois, Nrois).
    1) corr_matrix
    2) mi_matrix
    3) gc_matrix
    4) dynamic_matrix
    """
    combined_tensor = np.stack([corr_matrix, mi_matrix, gc_matrix, dynamic_matrix], axis=0)
    return combined_tensor

# ----------------------------
#  Memory Usage Helper
# ----------------------------
def estimate_memory_usage(num_subjects, matrix_shape=(4, 116, 116), dtype=torch.float32):
    """
    Estimate memory usage in GB if you stored all subject tensors in memory.
    """
    bytes_per_element = torch.finfo(dtype).bits // 8
    total_elements = np.prod(matrix_shape) * num_subjects
    total_bytes = total_elements * bytes_per_element
    total_gb = total_bytes / (1024**3)  # Convert to GB
    return total_gb

# ----------------------------
#  Subject-Level Computation
# ----------------------------
def process_and_save_subject_matrices(subject_id, signals, group, output_dir):
    """
    For each subject:
      1) Compute correlation matrix
      2) Compute mutual information matrix
      3) Compute Granger causality matrix
      4) Compute dynamic connectivity variability matrix
      5) Combine into a single 4D tensor (4, ROI, ROI)
      6) Save to disk
    """
    # 1) Pearson correlation
    corr_matrix = compute_correlation_matrix(signals)

    # 2) Mutual Information
    mi_matrix = compute_mutual_information_matrix(signals)

    # 3) Granger Causality
    gc_matrix = compute_granger_causality_matrix_parallel(signals)

    # 4) Dynamic Connectivity Variability
    dynamic_matrix = compute_dynamic_connectivity_variability(signals)

    # Combine all into one tensor of shape (4, ROI, ROI)
    combined_tensor = combine_matrices_to_tensor(corr_matrix, mi_matrix, gc_matrix, dynamic_matrix)

    # Save the tensor to disk
    file_name = f'{group}_tensor_{subject_id}.pt'
    tensor_path = os.path.join(output_dir, file_name)
    torch.save(combined_tensor, tensor_path)

    return tensor_path


#  Main Execution

In [2]:
# prompt: import drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
import pandas as pd
import torch
import numpy as np
import scipy.io as sio
from tqdm import tqdm
import logging

# --- Project configuration ---
project_dir = '/home/diego/Escritorio/santiago/1st_paper/116ROIs'
os.chdir(project_dir)

# Load CSV file
csv_path = os.path.join(project_dir, 'DataBaseSubjects.csv')
subjects_df = pd.read_csv(csv_path)

# Directory for tensor storage
output_dir = os.path.join(project_dir, 'TensorData')
os.makedirs(output_dir, exist_ok=True)

# Initialize storage dictionaries
tensor_file_paths = {'AD': [], 'CN': [], 'Others': []}

# Helper functions (assuming already defined above):
# - process_and_save_subject_matrices()
# - estimate_memory_usage()

subjects = subjects_df['SubjectID'].tolist()
groups = subjects_df['ResearchGroup'].tolist()

with tqdm(total=len(subjects_df), desc="Constructing tensors") as pbar:
    for subject_id, research_group in zip(subjects, groups):
        file_path = os.path.join(project_dir, 'ROISignals', f"ROISignals_{subject_id}.mat")

        if os.path.exists(file_path):
            mat_data = sio.loadmat(file_path)
            signals = mat_data['ROISignals']

            # Ensure signals shape is (Time, ROIs)
            if signals.shape[0] < signals.shape[1]:
                signals = signals.T

            # Limit signals to 116 ROIs if needed
            signals = signals[:, :116]

            # Group assignment
            if research_group == 'AD':
                group_label = 'AD'
            elif research_group == 'CN':
                group_label = 'CN'
            else:
                group_label = 'Others'

            try:
                tensor_path = process_and_save_subject_matrices(
                    subject_id, signals, group_label, output_dir
                )
                tensor_file_paths[group_label].append(tensor_path)

            except Exception as e:
                logging.error(f"Error processing subject {subject_id}: {e}")

        pbar.update(1)

print("\n✅ All tensors successfully computed, saved, and stored.")

# Memory estimation
total_gb = estimate_memory_usage(len(subjects_df))
print(f"🔹 Estimated memory usage for all tensors: {total_gb:.2f} GB")




Constructing tensors: 100%|██████████| 352/352 [1:15:35<00:00, 12.89s/it]


✅ All tensors successfully computed, saved, and stored.
🔹 Estimated memory usage for all tensors: 0.07 GB
